# Del 5: frekvensanalyse i Python

Denne delen handler om å anvende teorien fra de forrige delene i praksis. Vi kommer til å bruke Python, men dette kan også gjøres i andre programmeringsspråk, eksempelvis MATLAB eller R. 

## Nyttige biblioteker

Vi starter med å laste inn noen biblioteker som vi kommer til å bruke:

```python
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt



SciPy er en samling av biblioteker for vitenskapelig programmering i Python. Vi kommer til å bruke funksjoner fra 'scipy.fft' for å utføre fouriertransformasjonen. 'scipy.signal' inneholder også mange nyttige funksjoner for signalbehandling, som for eksempel filtre eller vindusfunksjoner, men de er utenfor omfanget av denne ressursen. 

Hvis du er ikke kjent med NumPy og Matplotlib fra før av, kan du lese mer om dem [her](https://numpy.org/) og [her](https://matplotlib.org/). Vi bruker numpy for å representere dataene våre som numpy arrays, og matplotlib for å plotte dem. Det skal også sies at NumPy har en egen FFT funksjon, så om man ønsker å bruke denne i stedet for SciPy sin, er det også mulig.

Hvis du skal gjennomføre fouriertransformasjonen på et eksisterende datasett som ligger i en fil, kan Pandas biblioteket være nyttig for å lese inn dataene:

```python
import pandas as pd
```

Du kan lese mer om Pandas [her](https://pandas.pydata.org/). 

## Fouriertransformasjonen (DFT-en) i Python

La oss nå ta for oss en enkel tidsserie som vi ønsker å utføre fouriertransformasjonen på. Vi definerer tidsserien slik:

```python
t_start = 0 # Starttid, sekunder
t_slutt = 50 # Slutttid, sekunder 

fs = 1000 # Samplingsfrekvens, Hz
Ts = 1/fs # Samplingsintervall, sekunder 

N = fs * (t_slutt-t_start) # Antall samplepunkter

t = np.linspace(t_start, t_slutt, N) # tidsakse

# én sinus med amplitude 1 og frekvens 50 Hz og én cosinus med amplitude 3 og frekvens 100 Hz:
data = np.sin(2*np.pi*50*t) + 3*np.cos(2*np.pi*100*t) 

```
```data``` er altså summen av en sinus og cosinus, med henholdsvis amplitude på 1 og frekvens på 50 Hz, og amplitude 3 og frekvens på 100 Hz. Legg merke til hvordan ```N``` er definert: vi har i utgangspunktet samplingsfrekvensen og lengden på signalet vårt i sekunder, og vi regner ut ```N``` ved å gange de sammen. Vi bestemmer altså ikke ```N``` selv, og dette er som oftest tilfellet med "ekte" data, eksempelvis fra feltmålinger. Vi har en eller annen måling som varer i en bestemt antall sekunder, og vi har en samplingsfrekvens som er satt av måleinstrumentet vårt, og vi får da antallet samplepunkter ved å gange disse verdiene sammen. 

Å ta selve FFT-en er veldig enkelt:

```python 
fouriertransformert_data = sp.fft.fft(data) # Utfører Fouriertransformasjonen
```

Vi skriver fft to ganger fordi vi henter en funksjon som heter ``fft()`` fra en pakke som heter `fft`, som da ligger i scipy.

``fouriertransformert_data`` er nå en numpy array som inneholder resultatet av DFT-en, beregnet av FFT-algoritmen. Den har samme lengde som ``data``, og hvert element i ``fouriertransformert_data`` er et komplekst tall.

Videre kan vi normalisere resultatet ved å dele på antall elementer i ``data``:

```python
fouriertransformert_data = fouriertransformert_data / N # Normaliserer fouriertransformasjonen
```
Da kommer vi til å få frekvenskomponenter som er i samme enhet som dataene våre, og er ikke avhengig av lengden på tidsserien.

### Amplitude og fase
Amplituden finner vi ved å ta absoluttverdien av hvert element i ``fouriertransformert_data``:

```python
amplituder = np.abs(fouriertransformert_data) # Finner amplitudene
```

Fasen finner vi ved å ta argumentet til hvert element i ``fouriertransformert_data``. Vi kan bruke funksjonen ``angle()`` fra numpy for å gjøre dette:

```python
faser = np.angle(fouriertransformert_data) # Finner fasene
```

## Plotting

Nå som vi har utført fouriertransformasjonen, kan vi plotte resultatet. Vi ønsker å plotte amplituden ved å plotte frekvensene på x-aksen, og amplitudene på y-aksen.
Vi må først lage en array som definerer frekvensene (x-aksen). Dette kan vi gjøre med funksjonen ``fftfreq()`` fra ``scipy.fft``:

```python
frekvenser = sp.fft.fftfreq(N, Ts) # Lager en array med frekvenser
```

Nå kan vi plotte amplituden mot frekvensene:

```python
plt.plot(frekvenser, amplituder) # Plotter amplituden mot frekvensene
plt.xlabel('Frekvens [Hz]')
plt.ylabel('Amplitude')
plt.show()
```

Om dataen vår er reell og vi vil kun plotte amplituden for positive frekvenser, kan vi avgrense plottet til å kun vise frekvensene opp til Nyquist-frekvensen, samt gange amplitudene med 2 for å kompensere for at vi har halvert antallet frekvenser:

```python
plt.plot(frekvenser, 2*amplituder) 
plt.xlim(0, fs/2) # Setter x-aksen til å gå fra 0 til halvparten av samplingsfrekvensen
plt.xlabel("Frekvens (Hz)")
plt.ylabel("Amplitude")
```

## Oppsummering

Hele koden ser da slik ut:

```python
# Importerer nødvendige biblioteker

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

t_start = 0 # Starttid, sekunder
t_slutt = 50 # Slutttid, sekunder 

fs = 1000 # Samplingsfrekvens, Hz
Ts = 1/fs # Samplingsintervall, sekunder 

N = fs * t_slutt # Antall samplepunkter

t = np.linspace(t_start, t_slutt, N) # tidsakse

# én sinus med amplitude 1 og frekvens 50 Hz og én cosinus med amplitude 3 og frekvens 100 Hz:
data = np.sin(2*np.pi*50*t) + 3*np.cos(2*np.pi*100*t) 

# Ta FFT-en:

fouriertransformert_data = sp.fft.fft(data) # Utfører fouriertransformasjonen
fouriertransformert_data = fouriertransformert_data / N # Normaliserer fouriertransformasjonen

amplituder = np.abs(fouriertransformert_data) # Finner amplitudene
faser = np.angle(fouriertransformert_data) # Finner fasene

# Plotting 

frekvenser = sp.fft.fftfreq(len(data), Ts) # Lager en array med frekvenser

plt.plot(frekvenser, amplituder) # Plotter amplituden mot frekvensene
plt.xlabel("Frekvens (Hz)")
plt.ylabel("Amplitude")
plt.show()

# Om dataen er reell og vi vil kun plotte amplituden for positive frekvenser:
plt.plot(frekvenser, 2*amplituder)
plt.xlim(0, fs/2) # Setter x-aksen til å gå fra 0 til halvparten av samplingsfrekvensen
plt.xlabel("Frekvens (Hz)")
plt.ylabel("Amplitude")
plt.show()
```


Plottet for amplituden av det negative og positive frekvensspekteret ser slik ut:

<center><img src="plots/p5_full.png" width="400px">


Og plottet for amplituden med kun positive frekvenser ser slik ut:

<center><img src="plots/p5_pos.png" width="400px">

Du kan bruke koden over som en mal for å utføre Fouriertransformasjonen på dine egne data og plotte amplituden, bare omdefiner ``data`` til å være din egen tidsserie. Det skal også sies at malen ovenfor er en veldig enkel implementasjon av fouriertransformasjonen, og det finnes mange flere muligheter for å tilpasse den til dine egne behov. Du kan eksempelvis legge til feilhåndteringer, transformere flere datasett, osv. 

Du kan også endre på parametrene i tidsserien, som samplingsfrekvensen, start- og sluttiden, og frekvensene til sinus og cosinus, for å se hvordan dette påvirker frekvensspekteret. Du kan til og med prøve å transformere funksjoner som ikke er sinus eller cosinus, eller funksjoner som er aperiodisk. Dette hjelper mye med intuisjonen om hvordan FFT-en fungerer, og hva den gir oss.

## Oppgaver

### **Oppgave 1: Feil i koden** 

Vi ønsker å plotte amplituden til Fouriertransformasjonen av en datasett med koden under:

```python
data_FFT = sp.fft.fft(data) 
frequencies = sp.fft.fftfreq(N, Ts)

plt.plot(frequencies, data_FFT)
```
Anta at alt av biblioteker er importert, og at dataene og variablene er definert. 

Hva er galt med koden over, og hvordan kan du fikse det? 

### **Oppgave 2: Din egen DFT**

**a)** Lag en funksjon som utfører DFT-en på et datasett. Funksjonen skal ha signaturen ``def DFT(data):`` hvor ``data`` er et numpy array. Funksjonen skal returnere en numpy array som inneholder resultatet av DFT-en. Bruk formelen for DFT-en som er gitt i forrige kapittel, altså: 

$$X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j2\pi \frac{kn}{N}}$$

Hvor $x[n]$ er dataene og $N$ er antall samplepunkter. Husk at lengden på $X[k]$ skal være lik lengden på $x[n]$.


*Hint:* Bruk gjerne numpy sin ``exp()`` funksjon for å regne ut $e^{-j2\pi \frac{kn}{N}}$. Det kan også være lurt å intialisere $X[k]$ som en numpy array med samme lengde som $x[n]$ med ``np.zeros(N, dtype=complex)``. Vi skriver ``dtype=complex`` for å fortelle numpy at vi ønsker at arrayen skal inneholde komplekse tall.


**b)** Tror du at din implementasjon av DFT-en er like rask som SciPy sin FFT funksjon? Hvorfor/hvorfor ikke? Gjerne test hypotesen din!

## *Fasit* 

### **Oppgave 1:** 
Koden plotter Fouriertransformasjonen rett fram, og ikke amplituden av den. For å plotte amplitude, må vi ta absoluttverdien av hvert element i ``data_FFT``. Vi kan også normalisere resultatet, slik at enhentene til amplituden samsvarer med enhetne i dataen. Koden kan fikses slik:

```python
data_FFT = sp.fft.fft(data)/N  
amplitudes = np.abs(data_FFT)
... 
```

### **Oppgave 2:**
**a)** Her er en mulig implementasjon av DFT-en:

```python
def DFT(data):
    N = len(data)
    X = np.zeros(N, dtype=complex)
    for k in range(N):
        for n in range(N):
            X[k] += data[n] * np.exp(-2j*np.pi*k*n/N)
    return X
```

Dette er da DFT-en implementert rett fram fra formelen. 

**b)** DFT-en vi har implementert her har preg av en "brute force" implementasjon, og er derfor mye tregere enn SciPy sin FFT funksjon. Som vi diskuterte i del 4 er FFT-algoritmen mye raskere enn DFT-en, ergo SciPy sin FFT kommer til å kjøre mye fortere for store datasett.